In [8]:
import os
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from src.Utils.K_Fold import K_Fold
from src.data_loader import load_data
from src.analysis.SHAP import shap_analysis
from objective_function import objective_adaboost
from src.model.train_model import get_X_y,train_model
from src.analysis.LIME import lime_sensitivity_analysis
from src.Optimiser.HOA.hoa_optimizer import hoa_optimizer
from src.Utils.balance_dataset_smote_enc import balance_with_smote_enc

In [2]:
DATA_PATH = (
    "D:\ML\B(SMOTE-ENC)#K-FOLD3#M(XGBC&RFC)#O(HOA)#A(SHAP[PLOTS])\data\data.xlsx"
)
data = load_data(DATA_PATH)
# Load your data (make sure to implement or import load_data function)

target_column = "Stress Level "
categorical_features = [1]  # 'Gender' column index in feature set

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\hesam\AppData\Local\Temp\ipykernel_12556\121831338.py:2: SyntaxWarning: invalid escape sequence '\M'
  "D:\ML\B(SMOTE-ENC)#K-FOLD3#M(XGBC&RFC)#O(HOA)#A(SHAP[PLOTS])\data\data.xlsx"


In [3]:
balanced_df = balance_with_smote_enc(
    file_path=r"D:\ML\B(SMOTE-ENC)#K-FOLD3#M(XGBC&RFC)#O(HOA)#A(SHAP[PLOTS])\data\Dataset-10MO-Yahyavi (Student Stress Level).csv",
    target_col="Stress Level ",
    categorical_cols=["Gender"],
    output_path="balanced_health_data.csv",
)

Before balancing:
 Stress Level 
4     14
2     12
7     11
6     11
1     10
3     10
5     10
9      8
10     7
8      7
Name: count, dtype: int64

After balancing:
 Stress Level 
6     14
7     14
10    14
3     14
2     14
9     14
1     14
5     14
8     14
4     14
Name: count, dtype: int64


In [4]:

# Encode Gender: let's map Male=0, Other=1
balanced_df["Gender"] = balanced_df["Gender"].map({"Male": 0, "Female": 1,"Other":2})

balanced_df = balanced_df.round(1)

print(balanced_df)

book = load_workbook(DATA_PATH)
if "Data after SMOTE-ENC" in book.sheetnames:
    book.remove(book["Data after SMOTE-ENC"])
    book.save(DATA_PATH)
with pd.ExcelWriter(DATA_PATH, engine="openpyxl", mode="a") as writer:
    balanced_df.to_excel(writer, sheet_name="Data after SMOTE-ENC", index=False)


    

     Age Gender  Blood Oxygen Level (%)  Body Temperature (°C)  \
0     16      0                    97.5                   37.4   
1     17      1                    99.6                   36.6   
2     17      0                    97.5                   36.6   
3     15      2                    96.3                   36.9   
4     16      0                    99.1                   37.1   
..   ...    ...                     ...                    ...   
135   17      0                    99.0                   36.6   
136   15      0                    98.0                   36.8   
137   17      1                    99.7                   37.3   
138   17      1                    98.2                   36.5   
139   15      1                    98.9                   37.1   

     Heart Rate (bpm)  Physical Activity Level (METs)  Overall Health Score  \
0                  71                             7.1                    84   
1                  88                            

In [5]:
X, y = get_X_y(balanced_df, target_col="Stress Level ")

In [6]:
# Apply K-Fold cross-validation
(
    X_train,
    X_test,
    y_train,
    y_test,
    K_Fold_Cross_Validation_Scores,
    combined_df,
) = K_Fold(X, y, n_splits=5)

K_Fold_Cross_Validation_Scores = pd.DataFrame(K_Fold_Cross_Validation_Scores)

# Save combined K-Fold data to Excel
book = load_workbook(DATA_PATH)
if "DATA after K-Fold" in book.sheetnames:
    book.remove(book["DATA after K-Fold"])
    book.save(DATA_PATH)
with pd.ExcelWriter(DATA_PATH, engine="openpyxl", mode="a") as writer:
    combined_df.to_excel(writer, sheet_name="DATA after K-Fold", index=False)


🔁 Fold 1 ------------------
  ADAboost → R2: -0.0960, RMSE: 9.9310

🔁 Fold 2 ------------------
  ADAboost → R2: 0.1961, RMSE: 5.4182

🔁 Fold 3 ------------------
  ADAboost → R2: 0.1265, RMSE: 4.8620

🔁 Fold 4 ------------------
  ADAboost → R2: 0.0799, RMSE: 6.6693

🔁 Fold 5 ------------------
  ADAboost → R2: -3.0267, RMSE: 9.5069

✅ K-Fold Cross-Validation completed.
Best fold index: 1, R2: 0.1961
✅ Combined DataFrame using original target column name:


In [10]:
singleModel_result = train_model(X_train, y_train, X_test, y_test)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Gender: category

In [ ]:
# 🧱 Build stacking model from parameters

from src.model.train_model import build_stacking_model , evaluate_model ,objective_function


# 🎯 Objective function for optimization


# 📊 Evaluate model and print metrics


# 🚀 Main script

# 🔹 Raw model
raw_model = build_stacking_model()
evaluate_model(
    raw_model,
    "Raw Stacking",
    X_train,
    X_test,
    y_train,
    y_test,
)
# 🔹 Optimized model
problem = {
    "obj_func": objective_function,
    "lb": [0.01, 3, 50, 0.5, 50, 3, 2, 1],
    "ub": [0.3, 10, 300, 1.0, 300, 10, 10, 5],
    "minmax": "min",
    "log_to_file": False,
}
# model = HHO.OriginalHHO(epoch=50, pop_size=20)
# best = model.solve(problem)
# print("\n✅ Best parameters found:", best.solution)
# print(f"✅ Best cross-validated accuracy: {-best.target:.4f}")
# xgb_params = best.solution[:4]
# rf_params = best.solution[4:]
# optimized_model = build_stacking_model(xgb_params, rf_params)
# evaluate_model(optimized_model, "Optimized Stacking")



ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Gender: category

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from mealpy.human_based import HHO

# 📦 Load dataset
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 🧱 Build stacking model from parameters
def build_stacking_model(xgb_params, rf_params):
    xgb = XGBClassifier(
        learning_rate=xgb_params[0],
        max_depth=int(xgb_params[1]),
        n_estimators=int(xgb_params[2]),
        subsample=xgb_params[3],
        eval_metric="logloss",
        random_state=42
    )

    rf = RandomForestClassifier(
        n_estimators=int(rf_params[0]),
        max_depth=int(rf_params[1]),
        min_samples_split=int(rf_params[2]),
        min_samples_leaf=int(rf_params[3]),
        random_state=42
    )

    stacked_model = StackingClassifier(
        estimators=[("xgb", xgb), ("rf", rf)],
        final_estimator=LogisticRegression(),
        cv=5
    )

    return stacked_model

# 🎯 Objective function for optimization
def objective_function(params):
    xgb_params = params[:4]
    rf_params = params[4:]
    model = build_stacking_model(xgb_params, rf_params)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy").mean()
    return -score  # Minimize negative accuracy

# 🚀 Main script
def main():
    problem = {
        "obj_func": objective_function,
        "lb": [0.01, 3, 50, 0.5, 50, 3, 2, 1],  # Lower bounds for XGB and RF
        "ub": [0.3, 10, 300, 1.0, 300, 10, 10, 5],  # Upper bounds
        "minmax": "min",
        "log_to_file": False,
    }

    model = HHO.OriginalHHO(epoch=50, pop_size=20)
    best = model.solve(problem)

    print("✅ Best parameters:", best.solution)
    print("✅ Best accuracy:", -best.target)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'mealpy'

In [ ]:
best_pos, best_RMSE, convergence = hoa_optimizer(
    objective_adaboost,  # our AdaBoost objective
    [50, 0.01],  # lower bounds: n_estimators, learning_rate
    [300, 1.0],  # upper bounds
    2,  # dim
    10,  # n_agents
    10,  # max_iter
    X_train_best,
    y_train_best,
    X_test_best,
    y_test_best,
)



🥾 Iter 1/10 - Best RMSE: 353.08348
🥾 Iter 2/10 - Best RMSE: 353.08348
🥾 Iter 3/10 - Best RMSE: 351.82757
🥾 Iter 4/10 - Best RMSE: 351.82757
🥾 Iter 5/10 - Best RMSE: 351.82757
🥾 Iter 6/10 - Best RMSE: 350.92794
🥾 Iter 7/10 - Best RMSE: 350.92794
🥾 Iter 8/10 - Best RMSE: 350.92794
🥾 Iter 9/10 - Best RMSE: 350.92794
🥾 Iter 10/10 - Best RMSE: 350.92794


In [ ]:
HOA_model_result = train_model(
    X_train_best, y_train_best, X_test_best, y_test_best, best_pos
)

d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))
d:\ML\Hesam= SCI-6MO-Yahyavi-Deadline=8-7-2025\src\Utils\getAllMetric.py:101: RuntimeWarning: overflow encountered in cosh
  return np.mean(np.log(np.cosh(y_pred - y_true)))


In [ ]:
# SHAP on the HOA model
sensitivity_df_shap, shap_values = shap_analysis(
    model=HOA_model_result["model"],  # assuming train_model returns dict with "model"
    X_train=X_train_best,
    y_train=y_train_best,
    X_test=X_test_best,
    y_test=y_test_best,
    save_path=DATA_PATH,  # save to same Excel file
    sheet_name="SHAP_Sensitivity",
)

  0%|          | 0/172 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
sensitivity_LIME = lime_sensitivity_analysis(
    model=HOA_model_result["model"],
    X_train=X_train_best,
    y_train=y_train_best,
    X_test=X_test_best,
    y_test=y_test_best,
    sample_index=5,
    epsilon=0.05,
)



📊 LIME Sensitivity for each feature:
Very Active Minutes > 29.00: 24.7143
5.03 < Tracker Distance <= 7.14: 11.9335
3.27 < Light Active Distance <= 4.78: 4.8021
Very Active Distance > 1.82: 2.8707
Sedentary Minutes <= 738.75: 8.2220
5.00 < Fairly Active Minutes <= 18.00: 13.5337
197.00 < Lightly Active Minutes <= 266.00: 6.0718
7133.50 < Steps <= 10132.75: 9.7169
Moderately Active Distance > 0.73: 11.6844


d:\Python\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(
d:\Python\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but AdaBoostRegressor was fitted with feature names
  warnings.warn(


In [ ]:





print("\nBest AdaBoost Params:", best_pos)
print("Best RMSE:", best_RMSE)